
### Casper's Ghost Kitchen

Select `Run All` to initialize this notebook.

#### What specifically does this notebook do

1. Create's a catalog, by default, `caspers`, where all of Casper's assets live.
2. Create's a schema, `simulator`, which houses Casper's dimensional data tables (brands, menus, categories, and items) and a Volume (`events`) for raw JSON events

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS :CATALOG